In [1]:
%pip install opendartreader
%pip install matplotlib
%pip install seaborn
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import OpenDartReader
import requests
import re
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings(action='ignore')

# 한글 설정
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='NanumBarunGothic')
mpl.rc('axes', unicode_minus=False)

api_key = '0b57d455b453f0686d32b94c7f8070838ba0e0eb'
dart = OpenDartReader(api_key)

### df1 가공

In [3]:
# 데이터프레임1(df1) 구성
# KIND 엑셀파일(.xlsx) 불러오기
filename1 = '/home/kmu/Edward/dart/ipo_data/공모기업현황.xlsx'
df1 = pd.read_excel(filename1, engine='openpyxl')
df1.head()

시장구분        회사명     신고서제출일       수요예측일정시작      수요예측일정종료         청약일정시작  \
0       NaN        NaN        NaT            NaN           NaN            NaN   
1  코스닥 상장추진  티쓰리엔터테인먼트 2022-09-29     2022-11-01  ~ 2022-11-02     2022-11-07   
2       NaN        NaN        NaT   ~ 2022-11-02           NaN   ~ 2022-11-08   
3  코스닥 상장추진        제이오 2022-09-23     2022-11-04  ~ 2022-11-07     2022-11-09   
4       NaN        NaN        NaT   ~ 2022-11-07           NaN   ~ 2022-11-10   

         청약일정종료                  납입일 확정공모가 공모금액(백만원)      상장예정일  상장주선인/지정자문인  
0           NaN                  NaN   NaN       NaN        NaT          NaN  
1  ~ 2022-11-08  2022-11-10 00:00:00     -         - 2022-11-17  미래에셋증권 주식회사  
2           NaN                  NaN   NaN       NaN        NaT          NaN  
3  ~ 2022-11-10  2022-11-14 00:00:00     -         - 2022-11-18      한국투자증권㈜  
4           NaN                  NaN   NaN       NaN        NaT          NaN

In [4]:
df1 = df1.iloc[1::2,:]
df1

시장구분        회사명     신고서제출일       수요예측일정시작      수요예측일정종료  \
1     코스닥 상장추진  티쓰리엔터테인먼트 2022-09-29     2022-11-01  ~ 2022-11-02   
3     코스닥 상장추진        제이오 2022-09-23     2022-11-04  ~ 2022-11-07   
5     코스닥 상장추진        저스템 2022-09-19     2022-10-13  ~ 2022-10-14   
7     코스닥 상장추진         산돌 2022-09-16     2022-10-12  ~ 2022-10-13   
9     코스닥 상장추진     삼성스팩7호 2022-09-15     2022-10-11  ~ 2022-10-12   
..         ...        ...        ...            ...           ...   
815   코스닥 상장추진   미래에셋벤처투자 2019-01-30     2019-02-27  ~ 2019-02-28   
817  유가증권 상장추진        드림텍 2019-01-29     2019-02-25  ~ 2019-02-26   
819   코스닥 상장추진   유안타제4호스팩 2019-01-25     2019-03-04  ~ 2019-03-05   
821   코스닥 상장추진     에코프로비엠 2019-01-18     2019-02-14  ~ 2019-02-15   
823   코스닥 상장추진        셀리드 2019-01-07     2019-01-29  ~ 2019-01-30   

            청약일정시작        청약일정종료                  납입일  확정공모가 공모금액(백만원)  \
1       2022-11-07  ~ 2022-11-08  2022-11-10 00:00:00      -         -   
3       2022-11-09  ~ 2022-11-10  2022-11-14 00:00:00      -         -   
5       2022-10-19  ~ 2022-10-20  2022-10-24 00:00:00      -         -   
7       2022-10-18  ~ 2022-10-19  2022-10-21 00:00:00      -         -   
9       2022-10-17  ~ 2022-10-18  2022-10-20 00:00:00      -         -   
..             ...           ...                  ...    ...       ...   
815     2019-03-07  ~ 2019-03-08  2019-03-12 00:00:00   4500     20250   
817     2019-03-04  ~ 2019-03-05  2019-03-07 00:00:00  13000     59092   
819     2019-03-11  ~ 2019-03-12  2019-03-14 00:00:00   2000      8200   
821     2019-02-21  ~ 2019-02-22  2019-02-26 00:00:00  48000    172800   
823     2019-02-11  ~ 2019-02-12  2019-02-14 00:00:00  33000     39600   

         상장예정일   상장주선인/지정자문인  
1   2022-11-17   미래에셋증권 주식회사  
3   2022-11-18       한국투자증권㈜  
5   2022-10-28   미래에셋증권 주식회사  
7   2022-10-27         KB증권㈜  
9   2022-10-26         삼성증권㈜  
..         ...           ...  
815 2019-03-15         KB증권㈜  
817 2019-03-14  엔에이치투자증권주식회사  
819 2019-03-21        유안타증권㈜  
821 2019-03-05         대신증권㈜  
823 2019-02-20         삼성증권㈜  

[412 rows x 12 columns]

In [5]:
cols = ['수요예측일정시작', '수요예측일정종료']
df1['수요예측일정'] =df1[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df1

시장구분        회사명     신고서제출일       수요예측일정시작      수요예측일정종료  \
1     코스닥 상장추진  티쓰리엔터테인먼트 2022-09-29     2022-11-01  ~ 2022-11-02   
3     코스닥 상장추진        제이오 2022-09-23     2022-11-04  ~ 2022-11-07   
5     코스닥 상장추진        저스템 2022-09-19     2022-10-13  ~ 2022-10-14   
7     코스닥 상장추진         산돌 2022-09-16     2022-10-12  ~ 2022-10-13   
9     코스닥 상장추진     삼성스팩7호 2022-09-15     2022-10-11  ~ 2022-10-12   
..         ...        ...        ...            ...           ...   
815   코스닥 상장추진   미래에셋벤처투자 2019-01-30     2019-02-27  ~ 2019-02-28   
817  유가증권 상장추진        드림텍 2019-01-29     2019-02-25  ~ 2019-02-26   
819   코스닥 상장추진   유안타제4호스팩 2019-01-25     2019-03-04  ~ 2019-03-05   
821   코스닥 상장추진     에코프로비엠 2019-01-18     2019-02-14  ~ 2019-02-15   
823   코스닥 상장추진        셀리드 2019-01-07     2019-01-29  ~ 2019-01-30   

            청약일정시작        청약일정종료                  납입일  확정공모가 공모금액(백만원)  \
1       2022-11-07  ~ 2022-11-08  2022-11-10 00:00:00      -         -   
3       2022-11-09  ~ 2022-11-10  2022-11-14 00:00:00      -         -   
5       2022-10-19  ~ 2022-10-20  2022-10-24 00:00:00      -         -   
7       2022-10-18  ~ 2022-10-19  2022-10-21 00:00:00      -         -   
9       2022-10-17  ~ 2022-10-18  2022-10-20 00:00:00      -         -   
..             ...           ...                  ...    ...       ...   
815     2019-03-07  ~ 2019-03-08  2019-03-12 00:00:00   4500     20250   
817     2019-03-04  ~ 2019-03-05  2019-03-07 00:00:00  13000     59092   
819     2019-03-11  ~ 2019-03-12  2019-03-14 00:00:00   2000      8200   
821     2019-02-21  ~ 2019-02-22  2019-02-26 00:00:00  48000    172800   
823     2019-02-11  ~ 2019-02-12  2019-02-14 00:00:00  33000     39600   

         상장예정일   상장주선인/지정자문인                      수요예측일정  
1   2022-11-17   미래에셋증권 주식회사     2022-11-01 ~ 2022-11-02  
3   2022-11-18       한국투자증권㈜     2022-11-04 ~ 2022-11-07  
5   2022-10-28   미래에셋증권 주식회사     2022-10-13 ~ 2022-10-14  
7   2022-10-27         KB증권㈜     2022-10-12 ~ 2022-10-13  
9   2022-10-26         삼성증권㈜     2022-10-11 ~ 2022-10-12  
..         ...           ...                         ...  
815 2019-03-15         KB증권㈜     2019-02-27 ~ 2019-02-28  
817 2019-03-14  엔에이치투자증권주식회사     2019-02-25 ~ 2019-02-26  
819 2019-03-21        유안타증권㈜     2019-03-04 ~ 2019-03-05  
821 2019-03-05         대신증권㈜     2019-02-14 ~ 2019-02-15  
823 2019-02-20         삼성증권㈜     2019-01-29 ~ 2019-01-30  

[412 rows x 13 columns]

In [6]:
cols = ['청약일정시작', '청약일정종료']
df1['청약일정'] =df1[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df1

시장구분        회사명     신고서제출일       수요예측일정시작      수요예측일정종료  \
1     코스닥 상장추진  티쓰리엔터테인먼트 2022-09-29     2022-11-01  ~ 2022-11-02   
3     코스닥 상장추진        제이오 2022-09-23     2022-11-04  ~ 2022-11-07   
5     코스닥 상장추진        저스템 2022-09-19     2022-10-13  ~ 2022-10-14   
7     코스닥 상장추진         산돌 2022-09-16     2022-10-12  ~ 2022-10-13   
9     코스닥 상장추진     삼성스팩7호 2022-09-15     2022-10-11  ~ 2022-10-12   
..         ...        ...        ...            ...           ...   
815   코스닥 상장추진   미래에셋벤처투자 2019-01-30     2019-02-27  ~ 2019-02-28   
817  유가증권 상장추진        드림텍 2019-01-29     2019-02-25  ~ 2019-02-26   
819   코스닥 상장추진   유안타제4호스팩 2019-01-25     2019-03-04  ~ 2019-03-05   
821   코스닥 상장추진     에코프로비엠 2019-01-18     2019-02-14  ~ 2019-02-15   
823   코스닥 상장추진        셀리드 2019-01-07     2019-01-29  ~ 2019-01-30   

            청약일정시작        청약일정종료                  납입일  확정공모가 공모금액(백만원)  \
1       2022-11-07  ~ 2022-11-08  2022-11-10 00:00:00      -         -   
3       2022-11-09  ~ 2022-11-10  2022-11-14 00:00:00      -         -   
5       2022-10-19  ~ 2022-10-20  2022-10-24 00:00:00      -         -   
7       2022-10-18  ~ 2022-10-19  2022-10-21 00:00:00      -         -   
9       2022-10-17  ~ 2022-10-18  2022-10-20 00:00:00      -         -   
..             ...           ...                  ...    ...       ...   
815     2019-03-07  ~ 2019-03-08  2019-03-12 00:00:00   4500     20250   
817     2019-03-04  ~ 2019-03-05  2019-03-07 00:00:00  13000     59092   
819     2019-03-11  ~ 2019-03-12  2019-03-14 00:00:00   2000      8200   
821     2019-02-21  ~ 2019-02-22  2019-02-26 00:00:00  48000    172800   
823     2019-02-11  ~ 2019-02-12  2019-02-14 00:00:00  33000     39600   

         상장예정일   상장주선인/지정자문인                      수요예측일정  \
1   2022-11-17   미래에셋증권 주식회사     2022-11-01 ~ 2022-11-02   
3   2022-11-18       한국투자증권㈜     2022-11-04 ~ 2022-11-07   
5   2022-10-28   미래에셋증권 주식회사     2022-10-13 ~ 2022-10-14   
7   2022-10-27         KB증권㈜     2022-10-12 ~ 2022-10-13   
9   2022-10-26         삼성증권㈜     2022-10-11 ~ 2022-10-12   
..         ...           ...                         ...   
815 2019-03-15         KB증권㈜     2019-02-27 ~ 2019-02-28   
817 2019-03-14  엔에이치투자증권주식회사     2019-02-25 ~ 2019-02-26   
819 2019-03-21        유안타증권㈜     2019-03-04 ~ 2019-03-05   
821 2019-03-05         대신증권㈜     2019-02-14 ~ 2019-02-15   
823 2019-02-20         삼성증권㈜     2019-01-29 ~ 2019-01-30   

                           청약일정  
1       2022-11-07 ~ 2022-11-08  
3       2022-11-09 ~ 2022-11-10  
5       2022-10-19 ~ 2022-10-20  
7       2022-10-18 ~ 2022-10-19  
9       2022-10-17 ~ 2022-10-18  
..                          ...  
815     2019-03-07 ~ 2019-03-08  
817     2019-03-04 ~ 2019-03-05  
819     2019-03-11 ~ 2019-03-12  
821     2019-02-21 ~ 2019-02-22  
823     2019-02-11 ~ 2019-02-12  

[412 rows x 14 columns]

In [7]:
df1.drop(['수요예측일정시작','수요예측일정종료','청약일정시작','청약일정종료'], axis=1, inplace=True)
df1

시장구분        회사명     신고서제출일                  납입일  확정공모가 공모금액(백만원)  \
1     코스닥 상장추진  티쓰리엔터테인먼트 2022-09-29  2022-11-10 00:00:00      -         -   
3     코스닥 상장추진        제이오 2022-09-23  2022-11-14 00:00:00      -         -   
5     코스닥 상장추진        저스템 2022-09-19  2022-10-24 00:00:00      -         -   
7     코스닥 상장추진         산돌 2022-09-16  2022-10-21 00:00:00      -         -   
9     코스닥 상장추진     삼성스팩7호 2022-09-15  2022-10-20 00:00:00      -         -   
..         ...        ...        ...                  ...    ...       ...   
815   코스닥 상장추진   미래에셋벤처투자 2019-01-30  2019-03-12 00:00:00   4500     20250   
817  유가증권 상장추진        드림텍 2019-01-29  2019-03-07 00:00:00  13000     59092   
819   코스닥 상장추진   유안타제4호스팩 2019-01-25  2019-03-14 00:00:00   2000      8200   
821   코스닥 상장추진     에코프로비엠 2019-01-18  2019-02-26 00:00:00  48000    172800   
823   코스닥 상장추진        셀리드 2019-01-07  2019-02-14 00:00:00  33000     39600   

         상장예정일   상장주선인/지정자문인                      수요예측일정  \
1   2022-11-17   미래에셋증권 주식회사     2022-11-01 ~ 2022-11-02   
3   2022-11-18       한국투자증권㈜     2022-11-04 ~ 2022-11-07   
5   2022-10-28   미래에셋증권 주식회사     2022-10-13 ~ 2022-10-14   
7   2022-10-27         KB증권㈜     2022-10-12 ~ 2022-10-13   
9   2022-10-26         삼성증권㈜     2022-10-11 ~ 2022-10-12   
..         ...           ...                         ...   
815 2019-03-15         KB증권㈜     2019-02-27 ~ 2019-02-28   
817 2019-03-14  엔에이치투자증권주식회사     2019-02-25 ~ 2019-02-26   
819 2019-03-21        유안타증권㈜     2019-03-04 ~ 2019-03-05   
821 2019-03-05         대신증권㈜     2019-02-14 ~ 2019-02-15   
823 2019-02-20         삼성증권㈜     2019-01-29 ~ 2019-01-30   

                           청약일정  
1       2022-11-07 ~ 2022-11-08  
3       2022-11-09 ~ 2022-11-10  
5       2022-10-19 ~ 2022-10-20  
7       2022-10-18 ~ 2022-10-19  
9       2022-10-17 ~ 2022-10-18  
..                          ...  
815     2019-03-07 ~ 2019-03-08  
817     2019-03-04 ~ 2019-03-05  
819     2019-03-11 ~ 2019-03-12  
821     2019-02-21 ~ 2019-02-22  
823     2019-02-11 ~ 2019-02-12  

[412 rows x 10 columns]

In [8]:
df1 = df1.replace('-',np.nan)
df1

시장구분        회사명     신고서제출일        납입일    확정공모가  공모금액(백만원)  \
1     코스닥 상장추진  티쓰리엔터테인먼트 2022-09-29 2022-11-10      NaN        NaN   
3     코스닥 상장추진        제이오 2022-09-23 2022-11-14      NaN        NaN   
5     코스닥 상장추진        저스템 2022-09-19 2022-10-24      NaN        NaN   
7     코스닥 상장추진         산돌 2022-09-16 2022-10-21      NaN        NaN   
9     코스닥 상장추진     삼성스팩7호 2022-09-15 2022-10-20      NaN        NaN   
..         ...        ...        ...        ...      ...        ...   
815   코스닥 상장추진   미래에셋벤처투자 2019-01-30 2019-03-12   4500.0    20250.0   
817  유가증권 상장추진        드림텍 2019-01-29 2019-03-07  13000.0    59092.0   
819   코스닥 상장추진   유안타제4호스팩 2019-01-25 2019-03-14   2000.0     8200.0   
821   코스닥 상장추진     에코프로비엠 2019-01-18 2019-02-26  48000.0   172800.0   
823   코스닥 상장추진        셀리드 2019-01-07 2019-02-14  33000.0    39600.0   

         상장예정일   상장주선인/지정자문인                      수요예측일정  \
1   2022-11-17   미래에셋증권 주식회사     2022-11-01 ~ 2022-11-02   
3   2022-11-18       한국투자증권㈜     2022-11-04 ~ 2022-11-07   
5   2022-10-28   미래에셋증권 주식회사     2022-10-13 ~ 2022-10-14   
7   2022-10-27         KB증권㈜     2022-10-12 ~ 2022-10-13   
9   2022-10-26         삼성증권㈜     2022-10-11 ~ 2022-10-12   
..         ...           ...                         ...   
815 2019-03-15         KB증권㈜     2019-02-27 ~ 2019-02-28   
817 2019-03-14  엔에이치투자증권주식회사     2019-02-25 ~ 2019-02-26   
819 2019-03-21        유안타증권㈜     2019-03-04 ~ 2019-03-05   
821 2019-03-05         대신증권㈜     2019-02-14 ~ 2019-02-15   
823 2019-02-20         삼성증권㈜     2019-01-29 ~ 2019-01-30   

                           청약일정  
1       2022-11-07 ~ 2022-11-08  
3       2022-11-09 ~ 2022-11-10  
5       2022-10-19 ~ 2022-10-20  
7       2022-10-18 ~ 2022-10-19  
9       2022-10-17 ~ 2022-10-18  
..                          ...  
815     2019-03-07 ~ 2019-03-08  
817     2019-03-04 ~ 2019-03-05  
819     2019-03-11 ~ 2019-03-12  
821     2019-02-21 ~ 2019-02-22  
823     2019-02-11 ~ 2019-02-12  

[412 rows x 10 columns]

In [9]:
df1.dropna()

시장구분         회사명     신고서제출일        납입일    확정공모가  공모금액(백만원)  \
13    코스닥 상장추진       탑머티리얼 2022-09-01 2022-10-07  30000.0    60000.0   
15    코스닥 상장추진        오에스피 2022-08-25 2022-10-07   8400.0    17270.0   
17    코스닥 상장추진       모델솔루션 2022-08-24 2022-09-29  27000.0    27000.0   
19    코스닥 상장추진   SK증권제8호스팩 2022-08-24 2022-09-29   2000.0     7000.0   
21    코스닥 상장추진  한화플러스제3호스팩 2022-08-19 2022-09-23   2000.0    10000.0   
..         ...         ...        ...        ...      ...        ...   
815   코스닥 상장추진    미래에셋벤처투자 2019-01-30 2019-03-12   4500.0    20250.0   
817  유가증권 상장추진         드림텍 2019-01-29 2019-03-07  13000.0    59092.0   
819   코스닥 상장추진    유안타제4호스팩 2019-01-25 2019-03-14   2000.0     8200.0   
821   코스닥 상장추진      에코프로비엠 2019-01-18 2019-02-26  48000.0   172800.0   
823   코스닥 상장추진         셀리드 2019-01-07 2019-02-14  33000.0    39600.0   

         상장예정일   상장주선인/지정자문인                      수요예측일정  \
13  2022-10-18       한국투자증권㈜     2022-09-27 ~ 2022-09-28   
15  2022-10-14         대신증권㈜     2022-09-27 ~ 2022-09-28   
17  2022-10-07         KB증권㈜     2022-09-19 ~ 2022-09-20   
19  2022-10-06         SK증권㈜     2022-09-20 ~ 2022-09-21   
21  2022-09-29       한화투자증권㈜     2022-09-14 ~ 2022-09-15   
..         ...           ...                         ...   
815 2019-03-15         KB증권㈜     2019-02-27 ~ 2019-02-28   
817 2019-03-14  엔에이치투자증권주식회사     2019-02-25 ~ 2019-02-26   
819 2019-03-21        유안타증권㈜     2019-03-04 ~ 2019-03-05   
821 2019-03-05         대신증권㈜     2019-02-14 ~ 2019-02-15   
823 2019-02-20         삼성증권㈜     2019-01-29 ~ 2019-01-30   

                           청약일정  
13      2022-10-04 ~ 2022-10-05  
15      2022-10-04 ~ 2022-10-05  
17      2022-09-26 ~ 2022-09-27  
19      2022-09-26 ~ 2022-09-27  
21      2022-09-20 ~ 2022-09-21  
..                          ...  
815     2019-03-07 ~ 2019-03-08  
817     2019-03-04 ~ 2019-03-05  
819     2019-03-11 ~ 2019-03-12  
821     2019-02-21 ~ 2019-02-22  
823     2019-02-11 ~ 2019-02-12  

[401 rows x 10 columns]

### df1 추출

In [10]:
# 사용하는 열만 추출
df1 = df1.loc[:, ['시장구분', '회사명', '확정공모가', '공모금액(백만원)', '상장예정일']]

# 엑셀에서 병합된 행 데이터값이 NaN이므로 삭제
df1.dropna(axis = 0, inplace = True)

# 상장예정일 19~22년만 사용
target_year = [2019, 2020, 2021, 2022]
target_month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df1.query('상장예정일.dt.year == @target_year', inplace = True)
df1.query('상장예정일.dt.month == @target_month', inplace = True)

# 상장예정일 형식 변환 및 정렬(KRX 제공 데이터와 통일)
df1['상장예정일'] = df1['상장예정일'].dt.strftime('%Y%m%d')
df1.sort_values(by = '상장예정일', ascending = True, inplace = True)

# 시장구분 명칭 변경
df1['시장구분'] = df1['시장구분'].apply(lambda x: '유가증권' if x == '유가증권 상장추진' else '코스닥')

# 타입 변경
df1['확정공모가'] = df1['확정공모가'].astype(int)
df1['공모금액(백만원)'] = df1['공모금액(백만원)'].astype(int)

# 인덱스 재설정
df1.reset_index(drop = True, inplace = True)

df1.tail()

# try except

시장구분        회사명  확정공모가  공모금액(백만원)     상장예정일
397  코스닥   한국제11호스팩   2000      10000  20221020
398  코스닥         핀텔   8900      17800  20221020
399  코스닥  하나금융25호스팩  10000      40000  20221020
400  코스닥  엔에이치스팩24호   2000       8600  20221020
401  코스닥       플라즈맵   7000      12397  20221021

In [11]:
# 스팩&리츠 이름은 삭제. 2,000원 주식도 스팩이 합병된 것이므로 삭제
df1 = df1[~df1['회사명'].str.contains('스팩|리츠|케이프이에스', na=False, case=False)]
df1 = df1[df1['확정공모가'] != 2000]

In [12]:
df1

시장구분       회사명  확정공모가  공모금액(백만원)     상장예정일
0     코스닥       셀리드  33000      39600  20190220
1     코스닥    에코프로비엠  48000     172800  20190305
2    유가증권       드림텍  13000      59092  20190314
3     코스닥  미래에셋벤처투자   4500      20250  20190315
5     코스닥     이지케어텍  12300      15990  20190322
..    ...       ...    ...        ...       ...
394   코스닥      오에스피   8400      17270  20221014
395   코스닥     탑머티리얼  30000      60000  20221018
396   코스닥       샤페론   5000      13735  20221019
398   코스닥        핀텔   8900      17800  20221020
401   코스닥      플라즈맵   7000      12397  20221021

[282 rows x 5 columns]

In [13]:
dart.company_by_name

<bound method OpenDartReader.company_by_name of <OpenDartReader.dart.OpenDartReader object at 0x7f61370737c0>>

## 2. 해당 공모주 가격밴드, 기관경쟁률, 의무보유확약

In [14]:
wish_prices_min = []
wish_prices_max = []
org_competition_rates = []
duty_rates = []

for i in range(len(df1)):
  # Dart 증권신고서 조회
  tdf = dart.list(corp = df1['회사명'][i])
  rcept = tdf[tdf['report_nm'].str.contains('증권신고서', na=False, case=False)]
  if len(rcept) > 0:
    rcept_no = rcept.iloc[0,:]['rcept_no']
    print(df1['회사명'][i], ', 보고서 번호: ', rcept_no)

    # 해당 보고서 번호(rcept_no)를 통해 url을 얻어내고, BeautifulSoup을 사용하여 필요한 데이터 추출
    urls = dart.sub_docs(rcept_no)
    url = urls.iloc[0].url
    print(url)

    # 공모가 밴드 하단, 상단
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', {'class': None})
    tr = table.find_all('tr')[1]
    td = tr.find_all('td')[1].get_text()
    wish_price_min = re.search(':(.+?)원', td).group(1).replace(',', '')
    wish_price_max = re.search('~(.+?)원', td).group(1).replace(',', '')
    wish_prices_min.append(wish_price_min)
    wish_prices_max.append(wish_price_max)

    print('희망공모가: ', wish_price_min, ' ~ ', wish_price_max)

    # 기관 경쟁률
    tds = soup.find_all('td')
    for j, td in enumerate(tds):
      if '경쟁률' in td:
        org_competition_rate = tds[j+7].get_text().split(':')[0].replace(',', '').strip()
        break

    # 의무보유 확약 비율
    for j, td in enumerate(tds):
      if '대비 비율' in td.get_text():
        duty_rate = tds[j+1].get_text().replace('%', '').strip()
        break

    for j, td in enumerate(tds):
      if '미확약' in td.get_text():
        no_duty = float(tds[j+14].get_text().replace(',', ''))
        sum_duty = float(tds[j+29].get_text().replace(',', ''))
        duty_rate = 100* (1 - no_duty/sum_duty)
        break

    org_competition_rates.append(org_competition_rate)
    duty_rates.append(duty_rate)
    print('경쟁률: ', org_competition_rate)
    print('의무보유확약: ', duty_rate, '%')
    print()
  else: 
    df1.drop(i, axis=0, inplace = True)
  
df1.reset_index(drop = True, inplace = True)

셀리드 , 보고서 번호:  20190201000197
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190201000197&dcmNo=6468234&eleId=1&offset=737&length=273785&dtd=dart3.xsd
희망공모가:   25000  ~   31000
경쟁률:  778.77
의무보유확약:  10.49 %

에코프로비엠 , 보고서 번호:  20220614000070
http://dart.fss.or.kr/report/viewer.do?rcpNo=20220614000070&dcmNo=8698845&eleId=1&offset=785&length=76955&dtd=dart3.xsd


AttributeError: 'NoneType' object has no attribute 'group'